# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [1]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [4]:
# load the dataset
path= "den_eigen_8.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12
0,1,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-7.151720e-01,-4.142136e-01,1.935432,2.139960,2.414214,5.051403
1,1,-2.442576,-2.236068,-2.000000,-1.325569,-1.000000,-1.000000,-1.000000e+00,-4.142136e-01,1.905424,2.236068,2.414214,4.862720
2,1,-2.236068,-2.236068,-2.236068,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000e+00,2.236068,2.236068,2.236068,5.000000
3,1,-2.475110,-2.106840,-2.000000,-1.641791,-1.532089,-0.979173,-3.472964e-01,-2.443955e-01,1.811645,1.879385,2.535777,5.099887
4,1,-2.510866,-2.171834,-1.971432,-1.552061,-1.323582,-0.952618,-4.376067e-01,-2.922416e-01,1.646544,2.137951,2.514007,4.913739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,0,-2.618034,-2.618034,-1.618034,-1.618034,-0.561553,-0.381966,-3.819660e-01,4.384472e-01,0.618034,0.618034,3.561553,4.561553
1428,0,-2.900523,-2.000000,-2.000000,-1.581807,-0.798880,-0.486482,-1.332394e-15,6.662604e-16,0.591077,0.912755,3.467932,4.795929
1429,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-3.212892e-16,3.212892e-16,0.486456,0.618034,3.713585,4.387619
1430,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-3.212892e-16,3.212892e-16,0.486456,0.618034,3.713585,4.387619


In [5]:
# Let's round up the columns to 8 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 8 decimals
df[cols_to_round] = df[cols_to_round].round(8)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12
0,1,-2.472834,-2.156712,-2.000000,-1.462598,-1.319479,-1.000000,-0.715172,-0.414214,1.935432,2.139960,2.414214,5.051403
1,1,-2.442576,-2.236068,-2.000000,-1.325569,-1.000000,-1.000000,-1.000000,-0.414214,1.905424,2.236068,2.414214,4.862720
2,1,-2.236068,-2.236068,-2.236068,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.236068,2.236068,2.236068,5.000000
3,1,-2.475110,-2.106840,-2.000000,-1.641791,-1.532089,-0.979173,-0.347296,-0.244395,1.811645,1.879385,2.535777,5.099887
4,1,-2.510866,-2.171834,-1.971432,-1.552061,-1.323582,-0.952618,-0.437607,-0.292242,1.646544,2.137951,2.514007,4.913739
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,0,-2.618034,-2.618034,-1.618034,-1.618034,-0.561553,-0.381966,-0.381966,0.438447,0.618034,0.618034,3.561553,4.561553
1428,0,-2.900523,-2.000000,-2.000000,-1.581807,-0.798880,-0.486482,-0.000000,0.000000,0.591077,0.912755,3.467932,4.795929
1429,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-0.000000,0.000000,0.486456,0.618034,3.713585,4.387619
1430,0,-2.618034,-2.230725,-1.874076,-1.618034,-0.482859,-0.381966,-0.000000,0.000000,0.486456,0.618034,3.713585,4.387619


# ChatGpt clasifier computation

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [7]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7282229965156795
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.69      0.71       141
           1       0.72      0.77      0.74       146

    accuracy                           0.73       287
   macro avg       0.73      0.73      0.73       287
weighted avg       0.73      0.73      0.73       287



### Decision tree

In [10]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.662020905923345
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.65      0.65       141
           1       0.66      0.68      0.67       146

    accuracy                           0.66       287
   macro avg       0.66      0.66      0.66       287
weighted avg       0.66      0.66      0.66       287



### Random forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=12)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.7770034843205574
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.72      0.76       141
           1       0.76      0.83      0.79       146

    accuracy                           0.78       287
   macro avg       0.78      0.78      0.78       287
weighted avg       0.78      0.78      0.78       287



In [16]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
        feature  importance
10  DEN_EIGEN11    0.190737
9   DEN_EIGEN10    0.114303
11  DEN_EIGEN12    0.111617
8    DEN_EIGEN9    0.087113


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [17]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.7346368715083799


In [18]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


SVM accuracy on full dataset: 0.797486033519553


Polynomial fit

In [19]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.7793296089385475


## Random forest  perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces an almost perfect score

In [21]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 1.0


In [22]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
         feature  importance
10  DEN_EIGEN11    0.185544
9   DEN_EIGEN10    0.124542
11  DEN_EIGEN12    0.113650
8    DEN_EIGEN9    0.090658
5    DEN_EIGEN6    0.078253
